<a href="https://colab.research.google.com/github/kenjan1974/Prediction-of-quality-of-Wine/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C06_%E7%B4%85%E9%85%92%E5%93%81%E8%B3%AA%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model    import LogisticRegression
from sklearn.neighbors       import KNeighborsClassifier
from sklearn.tree            import DecisionTreeClassifier
from sklearn.svm             import SVC
from xgboost                 import XGBClassifier

# 1. 用逗號分隔讀檔（不用 sep=';'）
df = pd.read_csv(
    'https://raw.githubusercontent.com/aniruddhachoudhury/'
    'Red-Wine-Quality/master/winequality-red.csv'
)

# 2. 清理欄名：去頭尾空白、空格換底線
df.columns = df.columns.str.strip().str.replace(' ', '_')

# 3. 去重、去 NA，再打散重設索引
df_train = (
    df.drop_duplicates()
      .dropna()
      .sample(frac=1, random_state=42)
      .reset_index(drop=True)
)

# 列出所有欄位名稱
print(df_train.columns.tolist())

# 查看每個欄位的 dtype
print(df_train.dtypes)

# 看前 5 筆
print(df_train.head())

# 4. 相關係數觀測
corr = df_train.corr()['quality'].drop('quality')
print("與 quality 相關係數前五：\n", corr.abs().sort_values(ascending=False).head(5))

# 5. 隨機森林特徵重要性
X = df_train.drop('quality', axis=1)
y = df_train['quality']
rfc = RandomForestClassifier(n_estimators=100, random_state=42).fit(X, y)
importances = pd.Series(rfc.feature_importances_, index=X.columns).sort_values(ascending=False)
print("隨機森林前五重要特徵：\n", importances.head(5))

# 6. 正規化比較
features = ['fixed_acidity','volatile_acidity','residual_sugar']
std = StandardScaler().fit_transform(df_train[features])
df_std = pd.DataFrame(std, columns=features).describe().loc[['min','max','mean','std']]

mm = MinMaxScaler().fit_transform(df_train[features])
df_mm = pd.DataFrame(mm, columns=features).describe().loc[['min','max','mean','std']]

print("StandardScaler 統計量：\n", df_std)
print("MinMaxScaler 統計量：\n", df_mm)

# 7. Baseline 跑 RFC
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rfc_baseline = RandomForestClassifier(n_estimators=200, random_state=42)
scores = cross_val_score(rfc_baseline, X_train, y_train, cv=5)
print("Baseline CV 分數：", scores)


['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
fixed_acidity           float64
volatile_acidity        float64
citric_acid             float64
residual_sugar          float64
chlorides               float64
free_sulfur_dioxide     float64
total_sulfur_dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object
   fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
0            7.7             0.620         0.04             3.8      0.084   
1            8.2             0.635         0.10             2.1      0.073   
2            8.4             0.370         0.43             2.3      0.063   
3            9.9             0.490         0.58             3.5      0.094   
4            6.3             

In [9]:
pip install tensorflow

In [11]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model    import LogisticRegression
from sklearn.neighbors       import KNeighborsClassifier
from sklearn.tree            import DecisionTreeClassifier
from sklearn.svm             import SVC
from xgboost                 import XGBClassifier

df = pd.read_csv(
    'https://raw.githubusercontent.com/aniruddhachoudhury/'
    'Red-Wine-Quality/master/winequality-red.csv'
)


df.columns = df.columns.str.strip().str.replace(' ', '_')


df_train = (
    df.drop_duplicates()
      .dropna()
      .sample(frac=1, random_state=42)
      .reset_index(drop=True)
)


X = df_train.drop('quality', axis=1)
y = df_train['quality']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# # 1. scikit-learn 基本分類器比較
# models = {
#     'LogisticRegression': LogisticRegression(max_iter=1000, class_weight='balanced'),
#     'KNN':                KNeighborsClassifier(),
#     'DecisionTree':       DecisionTreeClassifier(class_weight='balanced'),
#     'RandomForest':       RandomForestClassifier(n_estimators=200, class_weight='balanced'),
#     'SVC':                SVC(class_weight='balanced')
# }

# results = []
# for name, model in models.items():
#     scores = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')
#     results.append({
#         'model': name,
#         'mean_acc': scores.mean(),
#         'std_acc':  scores.std()
#     })

# df_results = pd.DataFrame(results).sort_values('mean_acc', ascending=False)
# print("—— scikit-learn 分類器比較 ——")
# print(df_results)


# # 2. GridSearchCV 調參（以 RandomForest 為例）
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth':    [None, 10, 20],
#     'min_samples_split': [2, 5]
# }
# gs = GridSearchCV(
#     RandomForestClassifier(class_weight='balanced', random_state=42),
#     param_grid,
#     cv=3,
#     scoring='accuracy',
#     n_jobs=-1
# )
# gs.fit(X, y)
# print("最佳參數：", gs.best_params_)
# print("最佳交叉驗證分數：", gs.best_score_)

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras import Sequential, layers
from tensorflow.keras.utils import to_categorical


# 3a. XGBoost 分類
le = LabelEncoder()
y_encoded = le.fit_transform(y)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
scores_xgb = cross_val_score(xgb, X, y_encoded, cv=5, scoring='accuracy')
print("XGBoost Accuracy:", scores_xgb.mean(), "+/-", scores_xgb.std())

# 3b. TensorFlow Keras 簡易全連接網路

# 轉成 one-hot（視多分類情況）
y_cat = to_categorical(y - y.min())  # 如果 quality 最小是3，就 -3 再做 one-hot
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

model = Sequential([
    layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(y_cat.shape[1], activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)
print("Keras Test Accuracy:", model.evaluate(X_test, y_test)[1])

# 3c. PyTorch 簡易全連接網路（示意）
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# 準備資料
# 資料轉換
X_t = torch.tensor(X.values, dtype=torch.float32)

# 重新 LabelEncode y
le_torch = LabelEncoder()
y_t = torch.tensor(le_torch.fit_transform(y), dtype=torch.long)
dataset = TensorDataset(X_t, y_t)
loader  = DataLoader(dataset, batch_size=32, shuffle=True)

# 模型
class Net(nn.Module):
    def __init__(self, in_dim, hidden, out_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.fc(x)

net = Net(X.shape[1], 64, len(y.unique()))
opt = optim.Adam(net.parameters())
loss_fn = nn.CrossEntropyLoss()

# 訓練一個 epoch
net.train()
for xb, yb in loader:
    opt.zero_grad()
    preds = net(xb)
    loss = loss_fn(preds, yb)
    loss.backward()
    opt.step()
print("PyTorch 一輪訓練完成，loss=", loss.item())


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:36:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:36:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:36:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:36:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:36:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

XGBoost Accuracy: 0.5901372910787932 +/- 0.010885405968173882
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.3273 - loss: 6.0907 - val_accuracy: 0.4037 - val_loss: 1.3068
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4418 - loss: 1.3515 - val_accuracy: 0.4954 - val_loss: 1.2620
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5034 - loss: 1.1982 - val_accuracy: 0.4587 - val_loss: 1.2405
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5017 - loss: 1.1402 - val_accuracy: 0.4037 - val_loss: 1.2428
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5334 - loss: 1.1554 - val_accuracy: 0.4862 - val_loss: 1.2427
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5493 - loss: 1.1584 - val_accuracy: 0.4587 - val_loss: 1.2517
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5280 - loss: 1.1581 - val_accuracy: 0.4679 - val_loss: 1.2778
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5257 - loss: 1.1311 - val_accuracy: 0.4312 - val_loss: 1.2378
Ep

1. **關係大的特徵：定義與觀察方法**
   我們可以用「與品質分數的絕對 Pearson 相關係數」或「模型（如隨機森林）的特徵重要性」來衡量「關係大」。

   ```python
   # 計算相關係數
   corr = df_train.corr()['quality'].drop('quality')
   corr_abs = corr.abs().sort_values(ascending=False)
   print("最相關的幾個特徵：")
   print(corr_abs.head(5))

   # 隨機森林特徵重要性
   from sklearn.ensemble import RandomForestClassifier
   X = df_train.drop('quality', axis=1)
   y = df_train['quality']
   rfc = RandomForestClassifier(n_estimators=100, random_state=42).fit(X, y)
   importances = pd.Series(rfc.feature_importances_, index=X.columns).sort_values(ascending=False)
   print("最重要的幾個特徵：")
   print(importances.head(5))
   ```

   * **常見結果**：`alcohol`、`sulphates`、`volatile_acidity`、`citric_acid`、`density` 通常名列前茅。

2. **可排除的多餘欄位**

   * 如果某特徵與品質相關性極低（絕對相關係數 < 0.05）且在模型中重要性也接近 0，就可考慮剔除。
   * 例如 `chlorides`、`citric_acid`（視實際排序而定）可能可以移除，以減少雜訊與計算成本。

3. **比較不同正規化方法**

   ```python
   from sklearn.preprocessing import StandardScaler, MinMaxScaler

   features = ['fixed_acidity','volatile_acidity','residual_sugar']
   # Standard
   std = StandardScaler().fit_transform(df_train[features])
   df_std = pd.DataFrame(std, columns=features).describe().loc[['min','max','mean','std']]
   # Min-Max
   mm = MinMaxScaler().fit_transform(df_train[features])
   df_mm = pd.DataFrame(mm, columns=features).describe().loc[['min','max','mean','std']]

   print("StandardScaler 統計量：\n", df_std)
   print("MinMaxScaler 統計量：\n", df_mm)
   ```

   * **Standardizer** 令資料平均為 0、標準差為 1；適合對「分佈近常態」的特徵。
   * **Min–Max** 壓縮到 \[0,1]；適合需要統一尺度或對「極端值」較不敏感的模型。

4. **不平衡標籤的影響與解法**

   * **影響**：模型偏向預測多數類別，中、小樣本類錯誤率高。
   * **解法**：

     * **過採樣**（`resample`／SMOTE）
     * **欠採樣**（隨機刪除多數類）
     * **調整類別權重**（`class_weight='balanced'`）

   ```python
   from sklearn.utils import resample
   # 以過採樣為例
   counts = df_train['quality'].value_counts()
   max_n = counts.max()
   df_list = []
   for q, n in counts.items():
       subset = df_train[df_train['quality']==q]
       df_list.append(resample(subset, replace=True, n_samples=max_n, random_state=42))
   df_balanced = pd.concat(df_list)
   print(df_balanced['quality'].value_counts())
   ```

5. **特徵工程示例**

   ```python
   df_fe = df_train.copy()
   # 1. 酸度比：固定酸度 / 揮發性酸度
   df_fe['acidity_ratio'] = df_fe['fixed_acidity'] / df_fe['volatile_acidity']
   # 2. 硫氧比：游離SO₂ / 總SO₂
   df_fe['sulfur_ratio'] = df_fe['free_sulfur_dioxide'] / df_fe['total_sulfur_dioxide']
   # 3. 酒精密度比：酒精含量 / 密度
   df_fe['alcohol_density_ratio'] = df_fe['alcohol'] / df_fe['density']

   print(df_fe[['acidity_ratio','sulfur_ratio','alcohol_density_ratio']].head())
   ```

   * 這些新變數往往能揭露「不同化學指標間的交互關係」，有助提升模型表現。

──


2. **欄位定義**（依照 Kaggle/UCI 文件）

   > 所有單位都以 g/dm³ 計，除非另註。

   * `fixed_acidity`：固定酸度（檸檬酸、蘋果酸等除揮發性酸外的總酸）
   * `volatile_acidity`：揮發性酸度（主要是醋酸，過高會產生醋味）
   * `citric_acid`：檸檬酸（少量有助於風味和新鮮感）
   * `residual_sugar`：殘留糖分（發酵後仍存在的糖，過高易甜膩）
   * `chlorides`：氯化物（鹽分量，過高會感覺鹹）
   * `free_sulfur_dioxide`：游離二氧化硫（抗氧化劑，過量會嗆鼻）
   * `total_sulfur_dioxide`：總二氧化硫（游離＋結合型）
   * `density`：密度（與糖分、酒精、溫度有關）
   * `pH`：酸鹼值（小於3 表示偏酸）
   * `sulphates`：硫酸鹽（促進防腐和風味）
   * `alcohol`：酒精含量（% v/v）
   * `quality`：品質分數（整數，0–10 分，主觀評分）

3. **特徵（features）分佈觀察**

   * **固定酸度 (fixed\_acidity)**：大多集中在 6–10；平均約 8.3，略偏右尾。

   * **揮發性酸度 (volatile\_acidity)**：多數落在0.2–0.6；平均約0.53，右偏（少數高醋酸樣本）。

   * **檸檬酸 (citric\_acid)**：大部分在0.0–0.6；平均約0.27，零或極低的比例也不少。

   * **殘留糖分 (residual\_sugar)**：多集中在0.9–2.5，平均約2；明顯右偏（最高可達15以上）。

   * **氯化物 (chlorides)**：主要落在0.03–0.10，平均約0.087，分佈窄且左偏。

   * **游離二氧化硫 (free\_sulfur\_dioxide)**：多在0–30 mg/dm³，平均約15，很長尾（最高上百）。

   * **總二氧化硫 (total\_sulfur\_dioxide)**：多在50–150 mg/dm³，平均約120，同樣長尾。

   * **密度 (density)**：範圍極窄，約0.990–1.004 g/cm³，分佈近常態，平均≈0.996。

   * **pH**：集中在3.1–3.6，平均約3.31，分佈略偏左。

   * **硫酸鹽 (sulphates)**：主要落在0.4–0.8，平均約0.66，分佈偏左。

   * **酒精含量 (alcohol)**：範圍7–14%，平均約10.4%，分佈近常態但略右尾。

   > **總結**：多數特徵的分佈都呈現輕微偏態（尤其是殘糖、揮發性酸度、二氧化硫），而像密度、pH、酒精含量則較接近對稱分佈。

4. **標籤（quality）分佈**

   * 質量分數範圍：3–8 分

   * **5 分與 6 分** 最常見，合計佔全體樣本約70%

   * 3、4、7、8 分樣本相對稀少，其中 8 分極少（約0.3%）

   > **小結**：資料標籤明顯偏向中間分數，屬於高度不平衡的多分類問題。


1. **資料分析工作流程必備環節（不考慮準確度）**

   1. **定義問題**：釐清業務或研究目標，決定要做分類、迴歸或其他分析。
   2. **資料蒐集**：取得原始資料（CSV、資料庫、API…）。
   3. **資料檢視（Exploration）**：快速檢查欄位型別、缺失值、重複值與大致分佈。
   4. **資料清理**：處理缺失值、重複值、異常值，統一欄位名稱、資料格式。
   5. **特徵轉換**：必要時做類別編碼、數值標準化或拆／合新特徵。
   6. **資料切分**：依需求切成訓練集、測試集（或驗證集）。
   7. **Baseline 建立**：選個簡單模型／基準模型跑一次，檢查 pipeline 無誤。
   8. **模型訓練與預測**：用訓練集訓練、測試集預測。
   9. **評估與報告**：算指標、畫圖、撰寫結論。